# images data : train, validation, test

In [ ]:
import numpy as np

import tensorflow as tf
from tensorflow import set_random_seed
from tensorflow.python.client import device_lib

import keras
from keras.layers import *
# Input ,Dense, Dropout, Activation, LSTM, Add
# from keras.layers import Convolution2D, MaxPooling2D, Flatten, Reshape, concatenate, Add, merge, Lambda
from keras import backend as K
from keras.models import *
from keras.utils import *
#  np_utils
# from keras.utils import np_utils
from keras.models import Model,Sequential
from keras.layers.wrappers import TimeDistributed
from keras import metrics
from keras import optimizers 
from keras import losses
from keras.callbacks import EarlyStopping

from sklearn.preprocessing import scale, robust_scale, minmax_scale, maxabs_scale

import os, shutil

import cv2

import PIL
import PIL.Image as pi
from PIL import Image

from numpy.random import seed
# set_random_seed(3)
# seed(1)

In [ ]:
print(device_lib.list_local_devices())

In [ ]:
%pwd
%cd '' # good match 파일 경로로 이동
file_list_new = os.listdir()
file_list_new.sort()

In [ ]:
file_list = []
cor_v_num= 1
for idx,i in enumerate(file_list_new):
    if 'jpg' in i:
        if idx < 242*cor_v_num: # 개
            file_list.append(i)
# print(file_list2,len(file_list2))
# [v_file_set.append(v_file_list2[i:i+242]) for i in range(0, len(v_file_list2), 242)]
# v_images_set = np.asarray(v_file_set)

In [ ]:
img_info = []
images = []
new=[]
for idx,i in enumerate (file_list):
    if 'jpg' in i:
        if idx < 242 * cor_v_num: #200개
            img = cv2.imread('{}'.format(i), cv2.IMREAD_COLOR)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, dsize=(150, 150), interpolation=cv2.INTER_AREA) # 퍼센트로 줄이자
            img = cv2.normalize(img, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F) #scaling 방식 (255로 나누자)
            img_info.append(img)
        else:
            break
new =np.asarray(img_info)

In [ ]:
%cd '' # bad match 파일 경로로 이동
file_list_origin = os.listdir()
file_list_origin.sort()

In [ ]:
file_list=[]
wron_v_num=1
for idx,i in enumerate(file_list_origin):
    if 'jpg' in i:
        if idx < 242 * wron_v_num: # 200개
            file_list.append(i)

# print(file_list2,len(file_list2))
# [v_file_set.append(v_file_list2[i:i+242]) for i in range(0, len(v_file_list2), 242)]
# v_images_set = np.asarray(v_file_set)

In [ ]:
img_info= []

for idx,i in enumerate (file_list):
    if 'jpg' in i:
        if idx < 242 * wron_v_num: # 500
            img = cv2.imread('{}'.format(i), cv2.IMREAD_COLOR)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, dsize=(150, 150), interpolation=cv2.INTER_AREA) # 퍼센트로 줄이자
            img = cv2.normalize(img, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F) #scaling 방식 (255로 나누자)
            img_info.append(img)
        else:
            break

origin=np.asarray(img_info)

In [ ]:
images = []
v_file_set= []
v_images_set = []

images.extend(new)
images.extend(origin)

[v_file_set.append(images[i:i+242]) for i in range(0, len(images), 242)]
v_images_set = np.asarray(v_file_set)
# images = np.asarray(img_info)

In [ ]:
v_images_set.shape

In [ ]:
v_data=v_images_set

# music data

In [ ]:
import librosa
import numpy as np
import pandas as pd
import scipy
import librosa.display
import IPython.display as ipd
from scipy.io import wavfile
from scipy import signal
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

%cd '' # 음악 cutting 파일들이 있는 경로로 이동

In [ ]:
m_file_list = os.listdir()
m_file_list.sort()
len(m_file_list)

In [ ]:
m_file_list2=[]
m_file_set = []
for i in m_file_list:
    if 'wav' in i:
        m_file_list2.append(i)

# [m_file_set.append(m_file_list2[i:i+242]) for i in range(0, len(m_file_list2), 242)]
# len(m_file_set)

In [ ]:
m_data=[]

for idx,i in enumerate(m_file_list2):
    if idx < 2000:
        y, sr = librosa.load('{}'.format(i))
        label=librosa.feature.melspectrogram(y=y, sr=sr,hop_length=512)
#       norm_label=librosa.util.normalize(label, norm=1)
        label=minmax_scale(label)
        m_data.append(label)

m_data = np.asarray(m_data)

In [ ]:
m_data = m_data.reshape((m_data.shape[0],m_data.shape[2],m_data.shape[1]))
m_data.shape # (sample,timeseries,feature)

# label 데이터 만들기

In [ ]:
a = np.random.randint(low=1,high=2,size=550)
b = np.random.randint(low=0,high=1,size=1450)

label = []
label.extend(a)
label.extend(b)
label = np.float32(np.expand_dims(label, 1))
np.asarray(label).T

# model 학습

In [ ]:
from keras.applications.inception_v3 import InceptionV3
from keras.utils.training_utils import multi_gpu_model

v_input_shape = (242, 150, 150, 3)
m_input_shape = (1292,128)
v_input = Input(v_input_shape)
m_input = Input(m_input_shape)

base_model = InceptionV3(weights=None, include_top=False, pooling='max')
inception_layer = TimeDistributed(base_model)(v_input)
Drop_layers = keras.layers.Dropout(0.3)(inception_layer)
inception_layer2 = TimeDistributed(Dense(128))(Drop_layers)
LSTM_layer = LSTM(242)(inception_layer2)
Drop_layers2 = keras.layers.Dropout(0.3)(LSTM_layer)
output = Dense(128)(Drop_layers2)
v_model = Model(v_input,output)

m_model = Sequential()
m_model.add(LSTM(242, input_shape = m_input_shape, return_sequences=True))
m_model.add(Dropout(0.3))
m_model.add(Flatten())
m_model.add(Dense(128))

encoded_v = output
encoded_m = m_model(m_input)

def euclidean_distance(vecter1):
    x, y = vecter1
    return K.sqrt(K.sum(K.square(x - y), axis = 1, keepdims=True))

def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

def contrastive_loss(y_true, y_pred):
    margin = 3
    square_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(margin - y_pred, 0))

    return K.mean(y_true * square_pred + (1 - y_true) * margin_square)


distance = keras.layers.Lambda(euclidean_distance, output_shape = eucl_dist_output_shape)([encoded_v,encoded_m])
# cosine = keras.layers.dot([encoded_v,encoded_m],axes=-1, normalize=True)
# both = Lambda(L1_distance,output_shape = lambda x: x[0])([encoded_v,encoded_m])
# both = merge([encoded_v,encoded_m], mode=L1_distance , output_shape = lambda x: x[0])
# https://github.com/eriklindernoren/Keras-GAN/issues/83 Lambda 참조
siamese_net = Model(inputs=[v_input,m_input],outputs=distance) 


siamese_net = multi_gpu_model(siamese_net, gpus=8)
siamese_net.compile(loss = contrastive_loss,
              optimizer = optimizers.RMSprop(lr=0.001),
              metrics = ['accuracy'])

In [ ]:
# early_stopping = EarlyStopping(monitor='acc', patience=)
hist = siamese_net.fit([v_data,m_data],
                     label , 
                     epochs = 30, 
                     verbose = 1, 
                     batch_size = 8, 
                     shuffle = True, 
                     validation_split = 0.2)
#                      callbacks = [early_stopping])
print('## training loss and acc ##')
print(hist.history['loss'], '\n')
print(hist.history['acc'], '\n')

In [ ]:
# siamese_net.predict([test_set,m_data])
m_model.predict(m_data)

In [ ]:
# 모델 전체 저장 - 경로 지정
siamese_net.save('final_model.h5')

In [ ]:
# 모델의 weight를 저장하기 - 경로 지정
siamese_net.save_weights('final_model_weight.h5')
print("Saved model to disk")

In [ ]:
# v_model만 저장 - 경로 지정
v_model.save('v_model.h5')

In [ ]:
# v_model의 weight를 저장하기 - 경로 지정
v_model.save_weights('v_model_weight.h5')
print("Saved model to disk")

In [ ]:
# m_model만 저장 - 경로 지정
m_model.save('m_model.h5')

In [ ]:
# m_model의 weight를 저장하기 - 경로 지정
m_model.save_weights('m_model_weight.h5')
print("Saved model to disk")

In [ ]:
# 모델 확인
siamese_net.summary()

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, loss, 'bo', label='Training loss')

plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
# 모델을 json파일로 저장하기
model_json = siamese_net.to_json()
with open("model.json", "w") as json_file : 
    json_file.write(model_json)

# 모델의 weight를 저장하기
seiamese_net.save_weights("model.h5")
print("Saved model to disk")


In [ ]:
# # 저장된 모델 불러오기
# from keras.models import model_from_json
# json_file = open("model.json", "r") 
# loaded_model_json = json_file.read() 
# json_file.close() 
# loaded_model = model_from_json(loaded_model_json)

# # 불러온 모델에 weight 적용하기
# loaded_model.load_weights("model.h5") 
# print("Loaded model from disk")

# # 모델에 weight 적용시켜 모델 돌리기

# loaded_model.compile(loss="binary_crossentropy", optimizer="rmsprop", metrics=['accuracy'])

# # model evaluation
# score = loaded_model.evaluate(X,Y,verbose=0)

# print("%s : %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

In [ ]:
train_generator = train_datagen.flow_from_directory(
        'data/train',
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary')